In [1]:
import pandas as pd
from vigipy import gps, bcpnn, lasso
from vigipy.utils.data_prep import convert, convert_binary, convert_multi_item
from vigipy.utils.expectations import test_dispersion

## Loading Data and Formatting for Analysis

The provided sample dataset `DYB_SUBSET.csv` is a selection of medical device report outcomes from the FDA MAUDE database. The data was automatically extracted from the original user report with a LLAMA3-based LLM and has only been minimally cleaned and processed. Unless you manually specify in the conversion functions, it is assumed that your input dataframe will have at least 3 columns: `AE` for the adverse event or problem, `name` for the brand name or generic name of the product used and `count` for the number of times that particular AE-device/drug combination occurs in the reporting system.

In [2]:
# Read and change column names to the default standards
df = pd.read_csv("DYB_SUBSET.csv")
df = df.rename({"Event": "AE", "REDUCED_BRAND": "name", "DATE_OF_EVENT": "date"}, axis=1)

# We only need a subset of the columns for processing
df = df[["name", "date", "AE", "count"]].copy()

## Processing Whole Data

The simplest approach to disproportionality analyses is to consider the entire corpus of reporting as a single collection of reporting trends with potentially disproportionate representation of particular drug/device-event combinations. To run a DA method on the entire corpus, it can be processed using `convert()` which will condense data into unique drug/device-event pairs and their counts.

The data container now available from the `convert()` function is compatible with the `gps()`, `bcpnn()`, `ror()` and `rfet()` analysis functions. For this analysis, we will use the multi-item gamma poisson shrinkage method `gps()`, discard any drug/device-event pairs with fewer than 3 reports, and will rank our results based on the lowers bound of the 5% posterior quantiles.

In [3]:
data = convert(df)
results = gps(data, min_events=3, decision_metric='rank', decision_thres=1, ranking_statistic='quantile', minimization_method="Nelder-Mead")
results.signals.head()

,Product,Adverse Event,Count,Expected Count,quantile,count/expected,product margin,event margin,fdr,FNR,Se,Sp,posterior_probability
0,ROTABLATOR GUIDEWIRE,Coating Loss,4.0,0.002642,120.112731,1513.750000,4.0,4.0,0.144542,0.848225,0.158898,0.853588,9.139363e-08
1,PRELUDE SHORT SHEATH INTRODUCER,Risk of Blood-Borne Pathogen Exposure,7.0,0.032700,79.915761,214.065657,22.0,9.0,0.139962,0.837552,0.747398,0.298457,5.272426e-12
2,UNIVERSAL MICROINTRODUCER KIT,Component Fragmentation,4.0,0.008588,79.629661,465.769231,13.0,4.0,0.142006,0.848082,0.162215,0.851586,4.824520e-07
3,BIOSENSE WEBSTER BRAIDED GUIDING SHEATH,Failure to Follow Precautions,5.0,0.016515,78.866729,302.750000,20.0,5.0,0.141106,0.849980,0.427330,0.585248,1.095973e-08
4,"6F/.070"" EXTRA LARGE LUMEN GUIDING CATHETER",Cardiovascular Complication,4.0,0.009909,74.050722,403.666667,10.0,6.0,0.149842,0.849105,0.152274,0.854281,6.463336e-07


# Fine-Tuning a DA Method

The base settings for all the exposed disproportionality analysis functions can be enhanced by using more appropriate prior assumptions depending on the use case. For example, instead of using the mantel-haentzel method of expected count estimation, we might consider a negative binomial model as being more appropriate. To do this, we could first calculate our dispersion and the accompanying alpha value by using the `test_dispersion()` function. 

In [4]:
dispersion_data = test_dispersion(data)
alpha = dispersion_data["alpha"] if dispersion_data["dispersion"] > 2 else 1

nb_results = bcpnn(data, expected_method='negative-binomial', method_alpha=alpha, min_events=3)
nb_results.signals.head()

g:\my drive\gitstuff\vigipy\src\vigipy\utils\expectations.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = results.params[0]


,index,Product,Adverse Event,Count,Expected Count,quantile,count/expected,product margin,event margin,fdr,FNR,Se,Sp
0,2409,RADIFOCUS II,Over-Pressurization,20.0,4.650502,2.631491,4.300611,200.0,25.0,0.159884,0.836633,0.697648,0.317422
1,2024,FAST-CATH,Leak,47.0,7.040947,2.307771,6.675238,445.0,76.0,0.160850,0.842351,0.631693,0.367570
2,121,GORE INTRODUCER SHEATH 22FR,Arterial Rupture,16.0,4.746063,2.257721,3.371215,239.0,20.0,0.015585,0.832410,0.039877,0.981377
3,2057,PSI,Leakage,23.0,5.898445,2.199331,3.899332,111.0,118.0,0.162607,0.843201,0.655812,0.342065
4,55,FAST-CATH,Air Leak,27.0,6.168218,2.132828,4.377277,445.0,38.0,0.008585,0.832523,0.026774,0.991936


# Using LASSO-LARS

As of version 1.4, vigipy now exposes the `lasso()` method of disproportionality analysis. To use this method, a different preprocessing approach is needed for the data. The products are treated as features of the LASSO model and are therefore processed into a binary occurrence matrix. `convert_binary()` takes the same raw dataframe as input and produces the appropriate product-drug/device binary matrices.

One the data container is ready, we can use `lasso()`. Lasso with the binary matrices can use either a vanilla LASSO, LASSO-LARS, or a LASSO with information criterion (AIC or BIC). You can also control confidence interval estimation through bootstrapping which is enabled by default.

In [5]:
bin_data = convert_binary(df)

#LASSO
ls_results = lasso(bin_data)

#LASSO-LARS
ll_results = lasso(bin_data, use_lars=True)

#LASSO-LARS + IC
lic = lasso(bin_data, use_IC=True, IC_criterion="aic")

lic.signals.head()

g:\my drive\gitstuff\vigipy\src\vigipy\utils\data_prep.py:129: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  event_df = event_df.groupby(by=event_df.columns, axis=1).sum()
g:\my drive\gitstuff\vigipy\src\vigipy\utils\data_prep.py:133: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  DC.product_features = prod_df.groupby(by=prod_df.columns, axis=1).sum()
C:\Users\shake\anaconda3\envs\vigipy\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\shake\anaconda3\envs\vigipy\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You

,Product,Adverse Event,LASSO Coefficient,CI Lower,CI Upper
168710,MONARCH IOL DELIVERY SYSTEM-CARTRIDGE,Device Stuck,1.0,0.225,1.0
13724,6F 11 CM IMPUT INTRODUCER KIT,Allergic Reaction,1.0,0.000,1.0
626600,SOFT TISSUE BIOPSY CO-AXIAL INTRODUCER NEEDLES,Unintended Movement,1.0,0.000,1.0
97030,ROTABLATOR GUIDEWIRE,Coating Loss,1.0,1.000,1.0
40847,CROSSFLEX LC 4.0X22/ 25MM,Balloon Rupture,1.0,0.000,1.0


# GLM-based LASSO

A fourth type of newly-support LASSO is the GLM-based LASSO. In this approach, rather than binarizing the event data in addition to the product data, only the product data is binarized while the event data retains its count information. This enables use of a negative-binomial family GLM to identify the relevant features in the product matrix.

In [ ]:
count_data = convert_binary(df, use_counts=True)
glm_results = lasso(count_data, use_glm=True, lasso_alpha=.001, lasso_thresh=0)
glm_results.signals.head()

g:\my drive\gitstuff\vigipy\src\vigipy\utils\data_prep.py:133: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  DC.product_features = prod_df.groupby(by=prod_df.columns, axis=1).sum()


# Turning any Model into a Time Series Observer

As mentioned eariler, the naive approach to DA is to consider the entire corpus simultaneously when identifying signals. A more nuanced approach might be to analyze the development of a signal over time. To provide this time-based analysis, vigipy exposes the `LongitudinalModel()` class which allows the selection of a time step (i.e. yearly, quarterly, monthly, etc) and splits the data into discrete groups. To do this, a `date` column must be available in the original input data before creating the `LongitudinalModel`. Once initialized, the class provides two major types of time-series analysis:

### Cumulative Analysis
The cumulative analysis method, exposed by `LongitudinalModel.run()` assumes that at each time group, the previously reported events and products all contribute to the overall pattern of potentially disproportionate reporting. Thus each time step rolls up all cumulative previous reports into the analysis. The most recent time step would therefore be equivalent to running the whole data at once as was done above.

### Disjoint Analysis
Another way to consider analysis DA time data is that signals may periodically spike, independent of the previous patterns or signals. `LongitudinalModel.run_disjoint()` processes only a single time step independent of the previous or future data. This provides less data at each step, but may be more sensitive to rapid changes in reported events.

### Handling Gaps
It is common that some DA datasets or vigilance networks will have very sparse periods of reporting. In these cases, the default behavior of the `LongitudinalModel` is to skip the time slice and represent the signal data as a `None`. This allows users to programatically maintain the same timestep structure in the output and identify those periods of sparsity. By setting `include_gaps` to `False`, the `run()` and `run_disjoint()` methods will instead skip these time steps entirely and not provide an empty representation.

In [ ]:
#Initialize the model with the data and a yearly time step.
LM = LongitudinalModel(df, "YE")
LM.date_groups.sum().head()

In [ ]:
LM.run_disjoint(bcpnn, include_gaps=False, expected_method='negative-binomial', method_alpha=alpha, min_events=3)
for timestamp, container in LM.results:
    print(container.signals.iloc[0])